In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
###### change current path
import os
path = "/content/gdrive/MyDrive/NUS_IRS_Project/"
os.chdir(path)

In [3]:
!ls

data  data_gathering  matching	Modelling  models  webapp_model.zip


In [4]:
# libraries import
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline

import re
import datetime
from datetime import date
from time import strptime

#import RAKE as rake
import operator

In [5]:
# First reading my job csv
job = pd.read_csv('data/dataset_job_sample.csv')

In [6]:
job.head()

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,210516002263,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8
1,find live infotech,UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...,0 - 0 yrs,Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,210516002391,Chennai,Work Based Onhome Based Part Time.,60.0,"1,50,000 - 2,50,000 P.A. 20000",2016-05-21 19:30:00 +0000,NaN,Marketing,d4c72325e57f89f364812b5ed5a795f0
2,Softtech Career Infosystem Pvt. Ltd,UG: Any Graduate - Any Specialization PG:Any P...,4 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,101016900534,Bengaluru,Pl/sql Developer - SQL,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,c47df6f4cfdf5b46f1fd713ba61b9eba
3,Onboard HRServices LLP,UG: Any Graduate - Any Specialization PG:CA Do...,11 - 15 yrs,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,81016900536,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,Accounts,115d28f140f694dd1cc61c53d03c66ae
4,Spire Technologies and Solutions Pvt. Ltd.,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,6 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,120916002122,Bengaluru,JAVA Technical Lead (6-8 yrs) -,4.0,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,a12553fc03bc7bcced8b1bb8963f97b4


In [7]:
job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              21996 non-null  object 
 1   education            20004 non-null  object 
 2   experience           21996 non-null  object 
 3   industry             21995 non-null  object 
 4   jobdescription       21996 non-null  object 
 5   jobid                22000 non-null  int64  
 6   joblocation_address  21499 non-null  object 
 7   jobtitle             22000 non-null  object 
 8   numberofpositions    4464 non-null   float64
 9   payrate              21903 non-null  object 
 10  postdate             21977 non-null  object 
 11  site_name            3987 non-null   object 
 12  skills               21472 non-null  object 
 13  uniq_id              22000 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 2.3+ MB


In [8]:
#removing nulls
job.drop(job.columns[[5, 9, 11, 13]], axis = 1, inplace = True) 

In [9]:
job.head()

,company,education,experience,industry,jobdescription,joblocation_address,jobtitle,numberofpositions,postdate,skills
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,Chennai,Walkin Data Entry Operator (night Shift),NaN,2016-05-21 19:30:00 +0000,ITES
1,find live infotech,UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...,0 - 0 yrs,Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,Chennai,Work Based Onhome Based Part Time.,60.0,2016-05-21 19:30:00 +0000,Marketing
2,Softtech Career Infosystem Pvt. Ltd,UG: Any Graduate - Any Specialization PG:Any P...,4 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,Bengaluru,Pl/sql Developer - SQL,NaN,2016-10-13 16:20:55 +0000,IT Software - Application Programming
3,Onboard HRServices LLP,UG: Any Graduate - Any Specialization PG:CA Do...,11 - 15 yrs,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,NaN,2016-10-13 16:20:55 +0000,Accounts
4,Spire Technologies and Solutions Pvt. Ltd.,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,6 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,Bengaluru,JAVA Technical Lead (6-8 yrs) -,4.0,2016-10-13 16:20:55 +0000,IT Software - Application Programming


In [10]:
job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              21996 non-null  object 
 1   education            20004 non-null  object 
 2   experience           21996 non-null  object 
 3   industry             21995 non-null  object 
 4   jobdescription       21996 non-null  object 
 5   joblocation_address  21499 non-null  object 
 6   jobtitle             22000 non-null  object 
 7   numberofpositions    4464 non-null   float64
 8   postdate             21977 non-null  object 
 9   skills               21472 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.7+ MB


In [11]:
job.drop(job[job['company'].isna()].index, axis=0, inplace=True)

In [13]:
replacements = {
   'joblocation_address': {
      r'(Bengaluru/Bangalore)': 'Bangalore',
      r'Bengaluru': 'Bangalore',
      r'Hyderabad / Secunderabad': 'Hyderabad',
      r'Mumbai , Mumbai': 'Mumbai',
      r'Noida': 'NCR',
      r'Delhi': 'NCR',
      r'Gurgaon': 'NCR', 
      r'Delhi/NCR(National Capital Region)': 'NCR',
      r'Delhi , Delhi': 'NCR',
      r'Noida , Noida/Greater Noida': 'NCR',
      r'Ghaziabad': 'NCR',
      r'Delhi/NCR(National Capital Region) , Gurgaon': 'NCR',
      r'NCR , NCR': 'NCR',
      r'NCR/NCR(National Capital Region)': 'NCR',
      r'NCR , NCR/Greater NCR': 'NCR',
      r'NCR/NCR(National Capital Region) , NCR': 'NCR', 
      r'NCR , NCR/NCR(National Capital Region)': 'NCR', 
      r'Bangalore , Bangalore / Bangalore': 'Bangalore',
      r'Bangalore , karnataka': 'Bangalore',
      r'NCR/NCR(National Capital Region)': 'NCR',
      r'NCR/Greater NCR': 'NCR',
      r'NCR/NCR(National Capital Region) , NCR': 'NCR'
       
   }
}

In [14]:
job.replace(replacements, regex=True, inplace=True)

In [15]:
job.head()

,company,education,experience,industry,jobdescription,joblocation_address,jobtitle,numberofpositions,postdate,skills
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,Chennai,Walkin Data Entry Operator (night Shift),NaN,2016-05-21 19:30:00 +0000,ITES
1,find live infotech,UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...,0 - 0 yrs,Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,Chennai,Work Based Onhome Based Part Time.,60.0,2016-05-21 19:30:00 +0000,Marketing
2,Softtech Career Infosystem Pvt. Ltd,UG: Any Graduate - Any Specialization PG:Any P...,4 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,Bangalore,Pl/sql Developer - SQL,NaN,2016-10-13 16:20:55 +0000,IT Software - Application Programming
3,Onboard HRServices LLP,UG: Any Graduate - Any Specialization PG:CA Do...,11 - 15 yrs,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,"Mumbai, Bangalore, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,NaN,2016-10-13 16:20:55 +0000,Accounts
4,Spire Technologies and Solutions Pvt. Ltd.,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,6 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,Bangalore,JAVA Technical Lead (6-8 yrs) -,4.0,2016-10-13 16:20:55 +0000,IT Software - Application Programming


In [17]:
from sklearn.impute import SimpleImputer
to_fill = ['education', 'skills']
for col in to_fill:
    imputer = SimpleImputer(strategy='most_frequent')
    job[col] = imputer.fit_transform(job[[col]])

In [18]:
job.isnull().sum()[job.isnull().sum()>0]

industry                   1
joblocation_address      497
numberofpositions      17532
postdate                  19
Education               1992
dtype: int64

In [19]:
job['Education'] = job['education'].str.split(' ')
job['Education'] = job['Education'].apply(lambda x: x[1] if len(x) > 1 else x[0])

job['Education'] = job['Education'].replace(('B.Tech/B.E.','Graduation','Other','-','Not','B.Tech/B.E.,','Postgraduate',
                                               'PG:CA','Diploma,','B.Com,','B.Pharma,','B.A,','BCA,','B.Sc,','MBA/PGDM','B.B.A,',
                                              'PG:Other','Doctorate:Doctorate','Post'),
                                              ('B.Tech','B.Tech','B.Tech','B.Tech','B.Tech','B.Tech','B.Tech',
                                              'CA','Diploma','B.Com','B.Pharma','B.A','BCA','B.Sc','MBA','BBA',
                                              'B.Tech','Doctorate','B.Tech'))

In [20]:
job.isnull().sum()[job.isnull().sum()>0]

industry                   1
joblocation_address      497
numberofpositions      17532
postdate                  19
dtype: int64

In [21]:
joblocation_imputer = SimpleImputer(strategy='most_frequent')
job['joblocation_address'] = joblocation_imputer.fit_transform(job[['joblocation_address']])

In [22]:
job.isnull().sum()[job.isnull().sum()>0]

industry                 1
numberofpositions    17532
postdate                19
dtype: int64

what we need:
company 
industry 
jd 
jobtitle
skills

job description

In [ ]:
### remove null
job.drop(job[job['jobdescription'].isna()].index, axis=0, inplace=True)

In [ ]:
job.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21996 entries, 0 to 21999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              21996 non-null  object 
 1   education            20004 non-null  object 
 2   experience           21996 non-null  object 
 3   industry             21995 non-null  object 
 4   jobdescription       21996 non-null  object 
 5   joblocation_address  21499 non-null  object 
 6   jobtitle             21996 non-null  object 
 7   numberofpositions    4464 non-null   float64
 8   postdate             21977 non-null  object 
 9   skills               21472 non-null  object 
dtypes: float64(1), object(9)
memory usage: 2.3+ MB


industy

In [ ]:
# analysis and clean industry
job['industry'].value_counts()

IT-Software / Software Services           9216
Education / Teaching / Training           1322
BPO / Call Centre / ITES                  1254
Banking / Financial Services / Broking    1238
Recruitment / Staffing                    1129
                                          ... 
Brewery / Distillery                         4
Medical Devices / Equipments                 4
Pulp and Paper                               2
Leather                                      1
Broadcasting                                 1
Name: industry, Length: 63, dtype: int64

In [ ]:
job['industry'].fillna('IT-Software / Software Services', inplace = True) 

In [ ]:
job['industry'].value_counts()

IT-Software / Software Services           9217
Education / Teaching / Training           1322
BPO / Call Centre / ITES                  1254
Banking / Financial Services / Broking    1238
Recruitment / Staffing                    1129
                                          ... 
Brewery / Distillery                         4
Medical Devices / Equipments                 4
Pulp and Paper                               2
Leather                                      1
Broadcasting                                 1
Name: industry, Length: 63, dtype: int64

In [ ]:
job.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21996 entries, 0 to 21999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              21996 non-null  object 
 1   education            20004 non-null  object 
 2   experience           21996 non-null  object 
 3   industry             21996 non-null  object 
 4   jobdescription       21996 non-null  object 
 5   joblocation_address  21499 non-null  object 
 6   jobtitle             21996 non-null  object 
 7   numberofpositions    4464 non-null   float64
 8   postdate             21977 non-null  object 
 9   skills               21472 non-null  object 
dtypes: float64(1), object(9)
memory usage: 2.3+ MB


job title

In [ ]:
job['jobtitle'] = job['jobtitle'].str.lower()

skills

In [ ]:
job['skills'].value_counts()

IT Software - Application Programming      5989
Sales                                      2893
ITES                                       1640
Teaching                                   1091
HR                                          928
Marketing                                   868
Accounts                                    860
Production                                  667
Medical                                     418
Financial Services                          413
IT Software - Other                         407
Engineering Design                          382
IT Software - Network Administration        372
IT Software - ERP                           354
IT Software - QA & Testing                  342
IT Software - eCommerce                     336
Site Engineering                            322
IT Software - DBA                           273
IT Software - Embedded                      262
IT Hardware                                 253
Journalism                              

In [ ]:
job['skills'].isnull().sum()

524

In [ ]:
job['skills'].fillna(' ', inplace = True)    ## fill no skills with empty string

In [ ]:
job['skills'].isnull().sum()

0

In [ ]:
job.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21996 entries, 0 to 21999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              21996 non-null  object 
 1   education            20004 non-null  object 
 2   experience           21996 non-null  object 
 3   industry             21996 non-null  object 
 4   jobdescription       21996 non-null  object 
 5   joblocation_address  21499 non-null  object 
 6   jobtitle             21996 non-null  object 
 7   numberofpositions    4464 non-null   float64
 8   postdate             21977 non-null  object 
 9   skills               21996 non-null  object 
dtypes: float64(1), object(9)
memory usage: 2.3+ MB


In [ ]:
x =  job.copy()
x.to_csv('data/EDA_job.csv', index=False)

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21996 entries, 0 to 21999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              21996 non-null  object 
 1   education            20004 non-null  object 
 2   experience           21996 non-null  object 
 3   industry             21996 non-null  object 
 4   jobdescription       21996 non-null  object 
 5   joblocation_address  21499 non-null  object 
 6   jobtitle             21996 non-null  object 
 7   numberofpositions    4464 non-null   float64
 8   postdate             21977 non-null  object 
 9   skills               21996 non-null  object 
dtypes: float64(1), object(9)
memory usage: 2.3+ MB
